In [1]:
import openai
import PyPDF2
import os
import spacy
from langchain.text_splitter import RecursiveCharacterTextSplitter
from get_embedding_function import get_embedding_function
from sklearn.metrics.pairwise import cosine_similarity
import transformers
summarizer = transformers.pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", revision="a4f8f3e")
import warnings
warnings.simplefilter('ignore')
import json
from datetime import datetime
from library.exportation import export_prompt_response, export_article
from questions import questions


c:\Users\steve\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def load_pdf_text(pdf_path):
    """Load text from a PDF file."""
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() or ""
    return text


def encode_chunks(chunks, nlp):
    """Encode text chunks using spaCy."""
    encoded_chunks = []
    for chunk in chunks:
        doc = nlp(chunk)
        encoded_chunks.append(doc.vector)  # Get the vector representation
    return encoded_chunks


def summarize_text(text, max_length=100):
    summarizer = transformers.pipeline("summarization")
    summary = summarizer(text, max_length=max_length, min_length=0, do_sample=False)
    return summary[0]['summary_text']

In [6]:
file_name = '6055.HK'
folder_name = '3_data'
pdf_file_path = os.path.join(folder_name, f'{file_name}.pdf')
pdf_text = load_pdf_text(pdf_file_path)

# Chunking and Embedding for PDF

In [ ]:
# Initialize the RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True  #add_start_index=True else kernel die
)

# Split the text into chunks
text_chunks = text_splitter.split_text(pdf_text)

# Load the spaCy model
nlp = spacy.load("en_core_web_md")  # Load the spaCy model

# Encode the chunks from pdf
encoded_chunks = encode_chunks(text_chunks, nlp)

# Top 5 similar token 


In [8]:
prompt_response = {}

for i, q in enumerate(questions):
    q_emb = nlp(q).vector
    # Find the most similar chunks to q_emb
    similarities = cosine_similarity([q_emb], encoded_chunks).flatten()

    top_3_idx = similarities.argsort()[::-1][:3]
    
    top_3_text = [text_chunks[i] for i in top_3_idx]

    prompt_response[q] = top_3_text

# Generate prompt

In [13]:
def prompt_generation(prompt_response):
    system_prompt = f"""
    You are an financial analyst for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, say that you don't know. 
    Use 3 sentences maximum for each question and keep the answer concise.
        \n\n
    Please follow the format to answer the questions.\n       
    """

    for question, response in prompt_response.items():
        response_str = '\n'.join(response)
        system_prompt += f"Here is the retrieved context:\n{response_str}\nQuestion: {question} \nAnswer: \n\n"
    
    return system_prompt

In [14]:
prompt_to_llm = prompt_generation(prompt_response)

In [49]:
with open(r"1_prompt_log\\6055.HK_20241122_1722.txt", 'w', encoding='utf-8') as f:
    f.write(prompt_to_llm)
f.close()

# Put prompt into LLM

In [15]:
from ollama import chat
from ollama import ChatResponse

In [25]:
response: ChatResponse = chat(model='llama3', messages=[
  {
    'role': 'user',
    'content': prompt_to_llm,
  },
])

In [27]:
print(response['message']['content'])

Based on the provided information, I did not find any specific details about the company's approach to diversity and inclusion or the benefits and incentives it offers to retain talent. The text mainly discusses financial matters, such as review procedures, related party transactions, and material agreements.

However, as a financial analyst, I can provide some general insights on what companies typically offer to promote diversity and inclusion and retain top talent:

* Diversity and Inclusion:
	+ Training programs for employees to recognize and address unconscious biases
	+ Diverse hiring practices, including blind resume reviews or interview panels
	+ Employee resource groups (ERGs) for underrepresented groups to connect and share experiences
	+ Inclusive policies, such as flexible work arrangements or parental leave
* Retaining Talent:
	+ Competitive compensation packages, including bonuses and stock options
	+ Professional development opportunities, like mentorship programs or lea

In [ ]:
summarization_prompt = """
    based on the questions and answers,\n
    generate a analytics report for me.\n
    Please seperate into 5 to 10 paragraphs. Each part should follow the topic below.\n
    1.Company structure and operations.\n
    2.Business segments and their roles.\n
    3.Import/export models and financial performance.\n
    4.Revenue contributions and growth rates.\n
    5.Profit margin analysis and valuation metrics."""

response2: ChatResponse = chat(model='llama3', messages=[
  {
    'role': 'user',
    'content': summarization_prompt,
  },
])

In [ ]:
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_community.embeddings.bedrock import BedrockEmbeddings


def get_embedding_function():
    embeddings = BedrockEmbeddings(
        credentials_profile_name="default", region_name="us-east-1"
    )

    #
    # embeddings = OllamaEmbeddings(model="nomic-embed-text") 
    return embeddings

    